In [1]:
import numpy as np
import tensorflow as tf
from utils_functions import *
from plan import *
import random
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
import datetime, os

In [ ]:
# controllo di star effettivamente usando la GPU

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

In [ ]:

dict_stati= load_file("./dizionario_stati")
plansLoaded=load_file("./plans")
#print(type(plansPickles))
#plansList=load_from_pickles("C:/Users/Francesco/Desktop/dataset/dataset/dizionario_stati",plansPickles)
#print(plansPickles)


In [ ]:
#ogni stato è un tensore con elementi dtype int8, questi stati vengono raggruppati in un altro tensore che rappresnta la
#variabile "states" del piano 
def build_vector(dict, states_list):
    l=len(dict)
    vector_states=[]
    for state in states_list:
        vector=np.array([0]*l,dtype=np.int8)
        #vector=tf.zeros(l,dtype=np.int8)
        for s in state:
            for key in dict.keys():
                if key==s:
                    vector[dict[key]-1]=1
                    break
        #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
        vector_states.append(vector) 
    r = np.array(vector_states)                          
    return r


#NON UTILIZZATO ADESSO
#shape di ogni singolo elemento (r) è (n x 340) con n che varia su ogni piano, raggruppati sulla base dei piani
def build_all_vectors(dict,plans_list):
    total=[]
    for plan in plans_list:
        r=build_vector(dict,plan.states)
        total.append(r)
    #r = tf.convert_to_tensor(total,dtype=None)                         
    return total
 
    
#gli stati vengono ordinati (in una lista) singolarmente con shape (1x340), non vengono raggruppati sulla base dei piani
#da utilizzare per autoencoder standard
def build_all_vectors1x340(dict, plans_list):
    l=len(dict)
    total=[]
    for plan in plans_list:
        for state in plan.states:
            vector=np.array([0]*l,dtype=np.int8)
            for s in state:
                for key in dict.keys():
                    if key==s:
                        vector[dict[key]-1]=1
                        break
            #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
            total.append(vector)
    total=np.array(total)        
    return total        

In [ ]:
#all_states_from_plans_list=build_all_vectors(dict_stati,plansLoaded)

## Preparazione dati

### Caricamento dataset statici

In [ ]:
test=load_file("./Dataset/set_test")
train=load_file("./Dataset/set_training")
validation=load_file("./Dataset/set_validation")
 

In [ ]:
test=np.array(test,dtype=np.int8)
train=np.array(train,dtype=np.int8)
validation=np.array(validation,dtype=np.int8)

## Modello

In [ ]:
# TODO: Cercare come migliorare l'autoencoder e come interpretare le metriche, 
# i test per il momento sono abbastanza a meno che non vuoi provare qualcosa di specifico sulla base di qello che trovi

#****************************************************************************************************

logdir = os.path.join("./TestLogs/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10),
    tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, write_graph=True, write_images=True)

]

#Cose da fare per migliorare la rete solo dopo aver fatto i primi tentativi con la rete proposta:
# * Provare ad aggiungere Regolarizzazione es. L1,L2 e dropout(solo nella fase di encoding)
# * Provare swish al posto di relu
# * Provare tf.keras.layers.BatchNormalization()(var_layer) 
# * Fare Hyperparameter Tuning  (ultima) www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams
#Salvare i risultati con ogni modifica fatta per scriverli nel report

input_size = 340 #costante

#Scelti casulmente al momento.... da rivedere come prima modifica forse
hidden_size = 170
#hidden_size2 = 85
code_size = 85

input_layer = Input(shape=(input_size,))
hidden_1 = Dense(hidden_size, activation='swish', kernel_initializer="he_uniform", )(input_layer)
#hidden_3 = Dense(hidden_size, activation='relu', kernel_initializer="he_uniform")(hidden_1)

code = Dense(code_size, activation='swish',kernel_initializer="he_uniform",)(hidden_1)
hidden_2 =Dense(hidden_size, activation='swish',kernel_initializer="he_uniform",)(code)
#hidden_4 =Dense(hidden_size, activation='relu',kernel_initializer="he_uniform")(hidden_2)

output_layer = Dense(input_size, activation='sigmoid')(hidden_2)

autoencoder = Model(input_layer, output_layer)
#variare learning_rate beta_1,beta_2, da fare per ultimo
#batch_size ???
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy',metrics=["accuracy","Precision","Recall"])
autoencoder.fit(x=train,y=train, epochs=500,batch_size=300000, validation_data=(validation,validation), callbacks=my_callbacks,
                      )

 #Gardare i risulati delle metriche su tensorbord                      

In [ ]:
#%tensorboard --logdir logsTf
%load_ext tensorboard
#Avvio da terminale: spostarsi nella cartella dove è presente la cartella di log, > tensorboard --logdir nomecartella

In [ ]:
autoencoder.summary()